In [ ]:
%pip install pfrl@git+https://github.com/voidful/pfrl.git
%pip install textrl==0.2.15
%pip install ipywidgets

In [ ]:
import pfrl
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead
import logging
import sys
import torch
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')
from NISQA.nisqa.NISQA_model import nisqaModel

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans")
model = AutoModelForSeq2SeqLM.from_pretrained("lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans")
model.eval()
model.cuda()

In [ ]:
sentiment = pipeline('sentiment-analysis',model="cardiffnlp/twitter-roberta-base-sentiment",tokenizer="cardiffnlp/twitter-roberta-base-sentiment",device=0,return_all_scores=True)

In [ ]:
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.CRITICAL)

In [ ]:
sentiment("dogecoin is bad")

In [ ]:
sentiment("dogecoin is bad")[0][0]['score']

In [ ]:
class MyRLEnv(TextRLEnv):
    def __init__(self, model, tokenizer, device="cuda", observation_input=[], max_length=100, compare_sample=2):
        super().__init__(model, tokenizer, observation_input, max_length, compare_sample)
        self.device = torch.device(device)
        self.model = model
        self.tokenizer = tokenizer
        self.observation_input = observation_input
        
        
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
        args = {
            'mode': 'predict_file',  # For example, 'predict_file', 'predict_dir', or 'predict_csv'
            'pretrained_model': '/home/b0990106x/TextRL/NISQA/weights/nisqa.tar',  # Name of the pretrained model file
            'deg': '/home/b0990106x/TextRL/NISQA/wav/test.wav',  # Path to the speech file if mode is predict_file
            'data_dir': None,  # Folder with speech files if mode is predict_dir
            'output_dir': '/home/b0990106x/TextRL/NISQA/result',  # Folder to output results.csv
            'csv_file': None,  # Name of the csv file if mode is predict_csv
            'csv_deg': None,  # Column name in csv with file names/paths if mode is predict_csv
            'num_workers': 0,  # Number of workers for PyTorch's DataLoader
            'bs': 1,  # Batch size for predicting
            'ms_channel': None,  # Audio channel in case of a stereo file, if needed
        }

        if args['mode'] == 'predict_file':
            if args['deg'] is None:
                raise ValueError('--deg argument with path to input file needed')
        elif args['mode'] == 'predict_dir':
            if args['data_dir'] is None:
                raise ValueError('--data_dir argument with folder with input files needed')
        elif args['mode'] == 'predict_csv':
            if args['csv_file'] is None:
                raise ValueError('--csv_file argument with csv file name needed')
            if args['csv_deg'] is None:
                raise ValueError('--csv_deg argument with csv column name of the filenames needed')
            if args['data_dir'] is None:
                args['data_dir'] = ''
        else:
                raise NotImplementedError('--mode given not available')
        args['tr_bs_val'] = args['bs']
        args['tr_num_workers'] = args['num_workers']
        
        nisqa = nisqaModel(args)
        prediction = nisqa.predict()
        reward = int(prediction['mos_pred'].iloc[0])
        return reward

    # def step(self, action):
    #     predicted, finish, predicted_str = self.predict(vocab_id=action)
    #     reward = self.get_reward(self.input_item, predicted, finish)
    #     # Assuming `action` is an index representing a choice of instruction or modification

    #     # Prepare the model input based on the chosen action
    #     # This involves selecting the instruction and possibly modifying voice features
    #     instruction, transcription, src_encodec_ids = self.prepare_input(action)

    #     # Generate voice output using the model
    #     # For simplicity, we're directly using a method that might encapsulate the model interaction
    #     layer_list = self.generate_voice(instruction, transcription, src_encodec_ids)

    #     # Convert model output to audio and evaluate it to calculate the reward
    #     encodec_code = convert_to_encode_code(self.tokenizer, layer_list)
    #     audio = synthesize_audio(encodec_code, self.device)
    #     reward = self.evaluate_audio(audio)  # This would be an external or predefined method

    #     # Check if the episode is done based on your criteria
    #     done = self.check_done()

    #     # Optionally update the environment state here

    #     return self.get_observation(), reward, done, {}
    
    # def predict(self, vocab_id):
    #     predicted_list = {}
    #     predicted_list_end = {}
    #     with torch.inference_mode():
    #         for i, (v_id, predicted, predicted_end) in enumerate(zip(vocab_id, self.predicted, self.predicted_end)):
    #             predicted_list_end[i] = False
    #             if not predicted_end:
    #                 pred_word = self.actions[v_id]
    #                 if pred_word in self.gen_stop_toks \
    #                         or len(pred_word) < 1 \
    #                         or len(predicted) > self.env_max_length:
    #                     predicted_list_end[i] = True
    #                     predicted_list[i] = [pred_word]
    #                 else:
    #                     predicted_list[i] = [pred_word]
    #             else:
    #                 predicted_list_end[i] = True
    #                 predicted_list[i] = ['']

    #         for i, (l, e) in enumerate(zip(predicted_list.values(), predicted_list_end.values())):
    #             self.predicted[i] = self.predicted[i] + l
    #             self.predicted_end[i] = e

    #         return self.predicted, all(self.predicted_end), [self.tokenizer.convert_tokens_to_string(i) for i in
    #                                                          self.predicted]
    # def reset(self):
    #     # Reset or initialize the environment state
    #     # This could involve loading or selecting a new dataset entry for instructions and voice features

    #     self.current_instruction, self.current_transcription, self.current_src_encodec_ids = self.select_new_instruction()

    #     # Prepare the initial observation based on the new instruction
    #     # This step depends on how you define an observation in your environment
    #     initial_observation = self.prepare_observation(self.current_instruction, self.current_transcription)

    #     return initial_observation


In [ ]:
# %pip install datasets
# from datasets import load_dataset

# dataset = load_dataset("lca0503/soxdata_encodec")
# dataset.save_to_disk("data")

In [ ]:
from datasets import load_from_disk
dataset = load_from_disk("data")

In [ ]:
print(dataset)

In [ ]:
input_text = [{}]
input_audio = ""

observation_list = [{'input':'i think dogecoin is'}]

In [ ]:
env = MyRLEnv(model, tokenizer, observation_input=observation_list, compare_sample=1)
actor = TextRLActor(env,model,tokenizer,optimizer='adamw',
                    temperature=0.8,
                    top_k=100,
                    top_p=0.85,)
agent = actor.agent_ppo(update_interval=50, minibatch_size=3, epochs=10,lr=3e-4)
print(actor.predict(observation_list[0]))

pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=3000,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='checkpoint', 
)
agent.load("./checkpoint/best")
print(actor.predict(observation_list[0]))